In [146]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
# from xgboost import XGBClassifier
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
%matplotlib inline
import warnings
warnings.filterwarnings('ignore')
sns.set_palette('cool')

In [147]:
### Trainings und Testdaten werden importiert ###
# training_data = pd.read_csv("/Users/maren/Downloads/train.csv")
# test_data = pd.read_csv('/Users/maren/Downloads/test.csv')

In [148]:
def data_engeneer_function(training_data, test_data):
    #### FEATURE ENGINEERING AND EXPLORATION ####
    passengerId = test_data['PassengerId'] # Aufheben um später die Daten einfach zu vergleichen

    # Die PassengerID trägt nicht zum Überleben oder Sterben bei, deshalb kann sie entfernt werden
    training_data.drop(labels='PassengerId', axis=1, inplace=True) 
    test_data.drop(labels='PassengerId', axis=1, inplace=True)
    # axis = 1: eine Spalte wird gedropped, inplace = True: die Aktion wird direkt auf das Element ausgeführt, 
    # sonst müsste man der Operation eine neue Variable zuordnen also new_test_data = test_data.drop(..)
    
    # Titel
    training_data['Title'] = training_data['Name'].apply(lambda x: x.split(',')[1]).apply(lambda x: x.split()[0])
    test_data['Title'] = test_data['Name'].apply(lambda x: x.split(',')[1]).apply(lambda x: x.split()[0])

    # Namenlänge
    training_data['Name_Len'] = training_data['Name'].apply(lambda x: len(x))
    test_data['Name_Len'] = test_data['Name'].apply(lambda x: len(x))

    # Das eigentliche Namenslabel wird entfernt, weil hieraus kein Mehrwert mehr entsteht
    training_data.drop(labels='Name', axis=1, inplace=True)
    test_data.drop(labels='Name', axis=1, inplace=True)
    
    # Auf einen Buchstaben kommt es nicht an, daher werden die Namen durch 10 geteilt um eine Ahnung zu bekommen

    test_data.Name_Len = (test_data.Name_Len/10).astype(np.int64)+1
    training_data.Name_Len = (training_data.Name_Len/10).astype(np.int64)+1
    
    
    ### AGE ###
    ## Auch hier macht es keinen Unterschied, ob jemand 61 oder 62 Jahre alt ist. Deshalb werden die Passagiere in 5 
    # Altersklassen eingeteilt

    # Taking care of null values in Age 
    full_data = pd.concat([training_data, test_data])

    # Null Ages in Training set (177 null values)
    train_age_mean = full_data.Age.mean()
    train_age_std = full_data.Age.std()
    train_age_null = training_data.Age.isnull().sum()
    rand_tr_age = np.random.randint(train_age_mean - train_age_std, train_age_mean + train_age_std, size=train_age_null)
    training_data['Age'][np.isnan(training_data['Age'])] = rand_tr_age
    training_data['Age'] = training_data['Age'].astype(int) + 1

    # Null Ages in Test set (86 null values)
    test_age_mean = full_data.Age.mean()
    test_age_std = full_data.Age.std()
    test_age_null = test_data.Age.isnull().sum()
    rand_ts_age = np.random.randint(test_age_mean - test_age_std, test_age_mean + test_age_std, size=test_age_null)
    test_data['Age'][np.isnan(test_data['Age'])] = rand_ts_age
    test_data['Age'] = test_data['Age'].astype(int)

    training_data.Age = (training_data.Age/15).astype(np.int64)
    test_data.Age = (test_data.Age/15).astype(np.int64) + 1
    
    
    ### FAMILY SIZE ###
    training_data['FamilySize'] = training_data['SibSp'] + training_data['Parch'] + 1
    test_data['FamilySize'] = test_data['SibSp'] + test_data['Parch'] + 1
    
    
    ### TRAVEL ALONE ###
    # Neue Spalte für isAlone: 0 = nicht alleine, 1 = reist alleine
    training_data['isAlone'] = training_data['FamilySize'].map(lambda x: 1 if x == 1 else 0)
    test_data['isAlone'] = test_data['FamilySize'].map(lambda x: 1 if x == 1 else 0)

    # Alte Spalten werden gelöscht
    training_data.drop(labels=['SibSp', 'Parch'], axis=1, inplace=True)
    test_data.drop(labels=['SibSp', 'Parch'], axis=1, inplace=True)

    ### TICKET LENGTH ###
    # Neue Spalte für die Länge des Tickets
    training_data['Ticket_Len'] = training_data['Ticket'].apply(lambda x: len(x))
    test_data['Ticket_Len'] = test_data['Ticket'].apply(lambda x: len(x))

    # Lösche Spalten des Tickets
    training_data.drop(labels='Ticket', axis=1, inplace=True)
    test_data.drop(labels='Ticket', axis=1, inplace=True) 

    ### CABIN ###
    
    # hasCabin Feature, da die Kabinennummer nicht viel über die Personen aussagt
    training_data['hasCabin'] = training_data.Cabin.notnull().astype(int)
    test_data['hasCabin'] = test_data.Cabin.notnull().astype(int)

    # drop der unnötigen Daten
    training_data.drop(labels='Cabin', axis=1, inplace=True)
    test_data.drop(labels='Cabin', axis=1, inplace=True)

    ### FARE ###
    test_data['Fare'][np.isnan(test_data['Fare'])] = test_data.Fare.mean()
    training_data.Fare = (training_data.Fare /20).astype(np.int64) + 1 
    test_data.Fare = (test_data.Fare /20).astype(np.int64) + 1 
    
    ### EMBARKED ###
    training_data['Embarked'] = training_data['Embarked'].fillna('S') # S ist der häufigste Einstiegsort
    return training_data, test_data

# training_data, test_data = data_engeneer_function(training_data, test_data)

In [149]:
### Inspection der Trainingsdaten ###
# training_data.head()

In [150]:
### Inspection der Testdaten ###
# test_data.head()

In [151]:
###  training_data.info()

In [152]:
# test_data.info()

In [153]:
### Bestimmung der Fehlenden Werte in den Daten ###
# training_data.isnull().sum()

In [154]:
def show_class_survival(data_frame):
    fx, axes = plt.subplots(1, 2, figsize=(15, 6))

    # figsize = (breite, höhe)
    axes[0].set_title("Pclass vs Frequency")
    axes[1].set_title("Pclass vise Survival rate")
    # countplot zählt wie oft ein Feature vorkam und stellt es in einem Balkendiagramm dar (eine Art Histogram)
    fig1_pclass = sns.countplot(data=data_frame, x='Pclass', ax=axes[0])
    fig2_pclass = sns.barplot(data=data_frame, x='Pclass',y='Survived', ax=axes[1])
    # data: Trainingsdaten, x,y: Feature aus den Daten, ax: die plt Axen von oben, für Titel und Größe
    print(data_frame[['Pclass', 'Survived']].groupby(['Pclass'], as_index=False).mean())

In [155]:
### Erkenntnis: ####
# Passagiere aus der 1. Klasse haben eine deutlich höhere Überlebenschance als Passagiere aus der 3. Klasse
## Gründe ? ##

In [156]:
### Name ###
# Der Titel oder die länge des Namens kann Aufschluss darüber geben, wie die Überlebenschancen stehen 
# Viele Namen = Zeichen von Wohlstand 
# show_class_survival(training_data)
# print (training_data[['Name_Len', 'Survived']].groupby(['Name_Len'], as_index=False).mean())
def show_title_survival(data_frame):
    
    fx, axes = plt.subplots(2, 1, figsize=(15, 10))
    axes[0].set_title("Title vs Frequency")
    axes[1].set_title("Title vise Survival rate")
    fig1_title = sns.countplot(data=data_frame, x='Title', ax=axes[0])
    fig2_title = sns.barplot(data=data_frame, x='Title',y='Survived', ax=axes[1])
    
    print (data_frame[['Title', 'Survived']].groupby(['Title'], as_index=False).mean())
    print (data_frame[['Title', 'Survived']].groupby(['Title'], as_index=False).count())#.mean())
# show_title_survival(training_data)

In [157]:
### Erkenntnis: ####
# Passagiere mit Sir, Lady, Ms im Titel haben höhere Überlebenschancen
# Mr hat die geringsten Überlebenschancen
# Längere Namen -> höhere Chancen
## Gründe ? ##

In [158]:
### Geschlecht und Alter###
# Frauen und Kinder zu erst?
def show_gender_survival (data_frame):
    # Geschlecht
    print(data_frame[['Sex', 'Survived']].groupby(['Sex'], as_index = False).mean())

    fx, axes = plt.subplots(1, 2, figsize=(15, 6))
    axes[0].set_title("Gender vs Frequency")
    axes[1].set_title("Gender vise Survival rate")
    fig1_gen = sns.countplot(data=data_frame, x='Sex', ax=axes[0])
    fig2_gen = sns.barplot(data=data_frame, x='Sex', y='Survived', ax=axes[1])
# show_gender_survival(training_data)

In [159]:
def show_age_survival(data_frame):
    # Alter

    # Wie oben festgestellt, gibt es viele fehlende Werte beim Alter, deshalb werden diese entfernt
    training_age_n = data_frame.Age.dropna(axis=0)

    fx, axes = plt.subplots(1, 2, figsize=(15,5))
    axes[0].set_title("Age vs frequency")
    axes[1].set_title("Age vise Survival rate")
    fig1_age = sns.distplot(a=training_age_n, bins=15, ax=axes[0], hist_kws={'rwidth':0.7})

    # Creating a new list of survived and dead

    pass_survived_age = data_frame[data_frame.Survived == 1].Age
    pass_dead_age = data_frame[data_frame.Survived == 0].Age

    axes[1].hist([data_frame.Age, pass_survived_age, pass_dead_age], bins=5, range=(0, 100), label=['Total', 'Survived', 'Dead'])
    axes[1].legend()
    plt.show()
    
    print(training_data[['Age', 'Survived']].groupby(['Age'], as_index = False).mean())
# show_age_survival(pd.read_csv("/Users/maren/Downloads/train.csv"))

In [160]:
### Erkenntnis: ####
# Frauen haben eindeutig die höhere Überlebensrate
# Die meisten Passagiere sind zwischen 20-40 Jahren alt
# In den Altersgruppen sind die Überlebensraten für junge Leute und 40 - 60 Jährige am Besten
## Gründe ? ##

In [161]:
### Familiengröße ###
# Wie schon beschrieben, kann dies ein Indikator für Wohlstand sein
def show_family_size_survival(data_frame):    
    # FamilySize = SibSp + Parch + 1


    fx, axes = plt.subplots(1, 2, figsize=(15,5))
    axes[0].set_title('Family Size counts')
    axes[1].set_title('Survival Rate vs Family Size')
    fig1_family = sns.countplot(x=data_frame.FamilySize, ax=axes[0], palette='cool')
    fig2_family = sns.barplot(x=data_frame.FamilySize, y=data_frame.Survived, ax=axes[1], palette='cool')
    
    print(data_frame[['FamilySize', 'Survived']].groupby(data_frame['FamilySize'], as_index=False).mean())
# show_family_size_survival(training_data)

In [162]:
### Erkenntnis: ####
# Die meisten Passagiere reisen alleine
# Passagiere aus Familien mit 3 - 4 Mitgliedern haben die höchsten Überlebenschancen
## Gründe ? ##

In [163]:
### Alleine Reisend ###
def show_travel_alone_survival(data_frame):
    # Plot
    fx, axes = plt.subplots(1, 2, figsize=(15, 6))
    fig1_alone = sns.countplot(data=data_frame, x='isAlone', ax=axes[0])
    fig2_alone = sns.barplot(data=data_frame, x='isAlone', y='Survived', ax=axes[1])
# show_travel_alone_survival(training_data)

In [164]:
### Ticket Länge ###
def show_ticket_lenght_survival(data_frame):
    # Plot
    fx, axes = plt.subplots(2, 1, figsize=(15, 10))
    axes[0].set_title("Ticket Length vs Frequency")
    axes[1].set_title("Length vise Survival rate")
    fig1_tlen = sns.countplot(data=data_frame, x='Ticket_Len', ax=axes[0])
    fig2_tlen = sns.barplot(data=data_frame, x='Ticket_Len',y='Survived', ax=axes[1])
# show_ticket_lenght_survival(training_data)

In [165]:
## In diesem Fall ist es nicht sicher, ob die länge des Tickets die Genauigkeit des Algorithmus verbessert,
## da die Unterschiede insignifkant sind
## Es hat sich allerdings ergeben, dass es die Performance verbessert!
## Nicht immer sind die wichtigen Daten so ersichtlich wie bei z.B. Geschlecht.

In [166]:
### Cabin###
def show_cabin_survival(data_frame):
    # plot
    fx, axes = plt.subplots(1, 2, figsize=(15, 6))
    fig1_hascabin = sns.countplot(data=data_frame, x='hasCabin', ax=axes[0])
    fig2_hascabin = sns.barplot(data=data_frame, x='hasCabin', y='Survived', ax=axes[1])
# show_cabin_survival(training_data)

In [167]:
### Fare ###
def show_fare_survival(data_frame):
    
    fx, axes = plt.subplots(1, 2, figsize=(15,5))
    fig1_fare = sns.distplot(a=data_frame.Fare, bins=15, ax=axes[0], hist_kws={'rwidth':0.7})
    fig1_fare.set_title('Fare vise Frequency')

    # Creating a new list of survived and dead

    pass_survived_fare = data_frame[data_frame.Survived == 1].Fare
    pass_dead_fare = data_frame[data_frame.Survived == 0].Fare

    axes[1].hist(x=[data_frame.Fare, pass_survived_fare, pass_dead_fare], bins=5, label=['Total', 'Survived', 'Dead'],\
            log=True)
    axes[1].legend()
    axes[1].set_title('Fare vise Survival')
    plt.show()
    
    print(data_frame[['Fare','Survived']].groupby(['Fare'], as_index = False).mean())
# show_fare_survival(training_data)

In [168]:
# Höhere Fare Kosten, höhere Überlebenschancen. 

In [169]:
def show_embarked_survival(data_frame):
    fx, axes = plt.subplots(1, 2, figsize=(15,5))
    axes[0].set_title('Embarked Counts')
    axes[1].set_title('Survival Rate vs Embarked')
    fig1_embarked = sns.countplot(x=data_frame.Embarked, ax=axes[0])
    fig2_embarked = sns.barplot(x=data_frame.Embarked, y=training_data.Survived, ax=axes[1])
# show_embarked_survival(training_data)

In [170]:
# Die meisten steigen in Southhampoton ein
# Die besten Chancen hat man, wenn man in Cherbourg einstieg (hier stiegen die meisten 1. Klasse Passagiere ein)

In [171]:
def prepare_data(training_data):
    
    X = training_data.iloc[:, 1:12].values
    y = training_data.iloc[:, 0].values
    
    ### LABEL ENCODER ###
    label_encoder_sex_tr = LabelEncoder()
    label_encoder_title_tr = LabelEncoder()
    label_encoder_embarked_tr = LabelEncoder()
    X[:, 1] = label_encoder_sex_tr.fit_transform(X[:, 1])
    X[:, 5] = label_encoder_title_tr.fit_transform(X[:, 5])
    X[:, 4] = label_encoder_embarked_tr.fit_transform(X[:, 4])
    
    # Splitting the dataset into training and test set

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.17)
    
    # Feature Scaling
    # Skaliert die Daten, sodass sie einheitlich nutzbar sind (zw -1 und 1)
    scaler_x = MinMaxScaler((-1,1))
    X_train = scaler_x.fit_transform(X_train)
    X_test = scaler_x.transform(X_test)
    return X_train, X_test, y_train, y_test
# X_train, X_test, y_train, y_test = prepare_data(training_data)